### 1. Bài toán nhà máy

 Một nhà máy sản xuất hai loại sản phẩm (I) và (II) từ hai loại nguyên liệu A và B. Biết rằng mỗi sản phẩm loại I cần 4 đơn vị nguyên liệu A và 2 đơn vị nguyên liệu B; mỗi sản phẩm loại (II) cần 2 đơn vị nguyên liệu A và 4 đơn vị nguyên liệu B. Khi bán một sản phẩm loại I lãi 8 đơn vị tiền, khi bán 1 sản phẩm loại (II) lãi 6 đơn vị tiền. Hãy lập kế hoạch sản xuất sao cho thu lãi nhiều nhất với số dự trữ nguyên liệu có hạn: 60 đơn vị nguyên liệu A và 48 đơn vị nguyên liệu B. 

In [5]:
from ortools.init.python import init
from ortools.linear_solver import pywraplp

def main():
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver("GLOP")

    # Khởi tạo 2 biến x, y và khoảng giá trị
    x = solver.NumVar(0, 20, "x")
    y = solver.NumVar(0, 20, "y")

    infinity = solver.infinity()

    # Khởi tạo ràng buộc : 4x + 2y <= 60
    constraint1 = solver.Constraint(-infinity, 60, "ct")
    constraint1.SetCoefficient(x, 4) # 4x
    constraint1.SetCoefficient(y, 2) # 2y

    # Khởi tạo ràng buộc : 2x + 4y <= 48
    constraint2 = solver.Constraint(-infinity, 48, "ct")
    constraint2.SetCoefficient(x, 2) # 2x
    constraint2.SetCoefficient(y, 4) # 4y

    # Khai báo hàm mục tiêu : f(x, y) = 8x + 6y -> max
    objective = solver.Objective()
    objective.SetCoefficient(x, 8)
    objective.SetCoefficient(y, 6)
    objective.SetMaximization()

    print(f"Solving with {solver.SolverVersion()}")
    result_status = solver.Solve()

    print(f"Status: {result_status}")
    if result_status != pywraplp.Solver.OPTIMAL:
        print("The problem does not have an optimal solution!")
        if result_status == pywraplp.Solver.FEASIBLE:
            print("A potentially suboptimal solution was found")
        else:
            print("The solver could not solve the problem.")
            return
            

    print("Solution:")
    print("Objective value =", objective.Value())
    print("x =", x.solution_value())
    print("y =", y.solution_value())

    print("Advanced usage:")
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")
    
main()

Solving with Glop solver v9.11.4210
Status: 0
Solution:
Objective value = 132.0
x = 11.999999999999996
y = 6.000000000000003
Advanced usage:
Problem solved in 0 milliseconds
Problem solved in 2 iterations


### 2. Bài toán kỹ sư

Có n kỹ sư và n công việc. Mỗi kỹ sư làm mỗi công việc sẽ được trả công lao động khác nhau. Bài toán yêu cầu thuê mỗi công nhân thực thi một công việc sao cho tổng tiền công phải trả là thấp nhất 